
# DTI Image Registration Using Diffusion Tensor Imaging ToolKit (DTI-TK)
## This notebook is my way of trying out DTI-TK image registration steps explained [here](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.HomePage)
## I created this notebook after [this](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.FirstRegistration) example tutorial. This notebook will perform the suggested steps by DTI-TK for correct image registration.

> should proceed by
>    * Reading the tutorials on data format, visualization and checking diffusivity unit
>    * Going through the detailed preprocessing tutorial
>    * Studying the detailed registration tutorial.

# BASIC PROCESSING AND VISUALIZATION 

## [Interoperability in DTI-TK with FSL](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Interoperability)
* Interoperability with FSL is straighforward using the `fsl_to_dtitk` script that converts a set of V[1-3] and L[1-3] dtifit files into the NIfTI tensor format that DTI-TK uses.

* To convert back to the FSL format, use TVEigenSystem, e.g.

        TVEigenSystem -in tensor.nii.gz -type FSL 

In [1]:
import os
from pathlib import Path
from pgimri.config import PROCESSED_DTI_FILENAME

In [2]:
# This is similar to running following commands in terminal
# export DTITK_ROOT=../dtitk
# export PATH=${PATH}:${DTITK_ROOT}/bin:${DTITK_ROOT}/utilities:${DTITK_ROOT}/scripts
dtitk_maindir = "../dtitk"
os.environ["DTITK_ROOT"] = dtitk_maindir
os.environ["PATH"] += f":{dtitk_maindir}/bin:{dtitk_maindir}/utilities:{dtitk_maindir}/scripts"
!which VolumeInfo

../dtitk/bin/VolumeInfo


> * In practice, we recommend the users to use fsl_to_dtitk, a script that takes a FSL-generated DTI eigensystem volumes (non-brain tissue already removed) and converts them into fully DTI-TK compatible DTI volumes, i.e., both in the correct format and correctly preprocessed.
> * The output from this program will be tensor_L{1,2,3}.nii.gz, the eigenvalues, and tensor_V{1,2,3}.nii.gz, the eigenvectors, following precisely the FSL naming convention.


> * **WARNING**: FSL also has an option that allows for a DTI volume to be saved on disk in matrix form similar to but NOT the same as the NIfTI tensor format. It stores the six independent matrix components using the "upper triangular" order rather than the "lower triangular" order specified in NIfTI standard! (See Section about CAMINO above for definition of upper and lower triangular) So DO NOT use this option if you want to use its output with DTI-TK!

In [3]:
!ls

 DTI_ITC.ipynb
 dti_L1.nii.gz
 dti_L2.nii.gz
 dti_L3.nii.gz
'DTI-TK image registration example.ipynb'
'DTI-TK image registration.ipynb'
 dti_V1.nii.gz
 dti_V2.nii.gz
 dti_V3.nii.gz
 ixi_aging_template_brain_mask.nii.gz
 ixi_aging_template.nii.gz
 reference_axial_tensor.png
 reference_coronal_tensor.png
 subjects_list.txt


In [4]:
# Convert FSL format to DTI-TK format for further processing
# base_filepath = f"../../data/DTI/processed_data/3_processed/S63080_HARSH_ITS/{PROCESSED_DTI_FILENAME}"
base_filepath = PROCESSED_DTI_FILENAME
!fsl_to_dtitk {base_filepath} # The output basename will be `<PROCESSED_DTI_FILENAME>_dtitk...`

Reading dti_L1.nii.gz ... Done in 0.009481s
Reading dti_V1.nii.gz ... Done in 0.02099s
Reading dti_L2.nii.gz ... Done in 0.00749s
Reading dti_V2.nii.gz ... Done in 0.021442s
Reading dti_L3.nii.gz ... Done in 0.006567s
Reading dti_V3.nii.gz ... Done in 0.021247s
Writing dti_dtitk.nii.gz ... Done in 0.182458s
Reading dti_dtitk.nii.gz ... Done in 0.039885s
Voxelwise scaling dti_dtitk.nii.gz by 1000 ... Done in 0.003107s
Writing dti_dtitk.nii.gz ... Done in 0.176866s
Reading dti_dtitk.nii.gz ... Done in 0.041417s
Computing the tensor Euclidean norm map ... Done in 0.003678s
Writing dti_dtitk_norm.nii.gz ... Done in 0.033514s
Reading dti_dtitk.nii.gz ... Done in 0.039636s
Reading dti_dtitk_norm_non_outliers.nii.gz ... Done in 0.002475s
Masking dti_dtitk.nii.gz by dti_dtitk_norm_non_outliers.nii.gz ... Done in 0.000813s
Writing dti_dtitk.nii.gz ... Done in 0.184281s
Reading dti_dtitk.nii.gz ... Done in 0.039006s
Converting dti_dtitk.nii.gz to be symmetric positive definite ... Done in 0.0711

## [VISUALIZING DTI IMAGES WITH DTI-TK](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.VisualizationTool)

### Below are reference views of how FA should look like:

Axial View            |  Cornonal View
:-------------------------:|:-------------------------:
![reference_axial_tensor.png](reference_axial_tensor.png)  |  ![reference_coronal_tensor.png](reference_coronal_tensor.png)

### Checking if tensors are correctly reconstructed
An important utility of the tensor glyph rendering described above is for checking the correctness of your tensor reconstruction. A common mistake during tensor reconstruction (the process of constructing tensor volumes from diffusion-weighted images) is using a gradient table that is not corrected for the difference between the coordinate frame in which the gradient table is defined and the one in which the image is defined. This type of erroneous reconstruction can not be detected with the RGB map (explained above) and need to be ruled out with the tensor glyph rendering.

The two steps are:

1. Check if the tensors are oriented correctly in the genu and the splenium of the corpus callosum in the axial view (shown above). The tensors' major axes should be clearly aligned along the boundary of the genu and the splenium.
2. Check if the tensors are oriented correctly in the internal capsule and the midbody of the corpus callosum in the coronal view (also shown above). Again, the tensors should be properly aligned with the boundary of these structures.

In [5]:
# Check the new volumes
dtitk_path = f"{base_filepath}_dtitk.nii.gz"

# Display basic volume info
!VolumeInfo {dtitk_path}

# Viewing the tensors using 3D ellipsoid glyphs
!TVglyphView -in {dtitk_path}

# Other ways to visualize
# !TVglyphView -in {dtitk_path} -scale 2 -view axial
# !TVglyphView -in {dtitk_path} -scale 2 -view coronal

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: LPI
Volume Info of dti_dtitk.nii.gz
size: 112x112x48, voxel size: 1.60714x1.60714x2.5, origin: [0, 0, 0]
Reading dti_dtitk.nii.gz ... Done in 0.041946s
press the key 'H' to print a list of the available control keystrokes
total number of colors = 262144
processing the selected axial slice ... done rendering 


# SPATIAL NORMALIZATION AND ATLAS CONSTRUCTION 

## [PREPROCESSING OF VOLUMES BEFORE REGISTRATION](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.BeforeReg)

### [**IMP**] Make Sure DTI Volumes are Using the Correct Diffusivity Unit. [More details here](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Diffusivity)
* **The unit used by DTI-TK is $10^{-3}mm^2.s^{-1}$**. With this unit, the mean diffusivity for CSF is around 3. You want to make sure your DTI volumes are stored in the correct unit.

* for diffusion-weighting, b-value of 800 s.mm-2 is used during the acquisition, and you input the number 800 as the b-value to your tensor reconstruction program, then the output DTI volume will have the unit of mm2.s-1.

* if your DTI volumes use the unit of mm2.s-1, then the multiplication factor should be 1,000. If your unit is m2.s-1, then the factor should be $10^9$.

In [6]:
# Unit of Diffusivity correction
# dtitk_scaled_path = dtitk_path.replace("_dtitk.nii.gz", "_dtitk_scaled.nii.gz")
# !TVtool -in {dtitk_path} -scale 1000 -out {dtitk_scaled_path}

In [7]:
# Check scaled file

#! [WARNING]: Do not run without scaling down. System will hang because of too heavy rendering.
# !TVglyphView -in {dtitk_scaled_path} -scale 0.001

### Remove Non-Brain Tissues (Already completed this step in `process-subject` function)
To do this, our recommended protocol is to use the excellent Brain Extraction Tool, aka BET, available as part of the FSL suite. Once you have computed the brain tissue mask and are satisfied with its quality, you can apply it to the corresponding DTI volume with the following command:

        TVtool -in tensor.nii.gz -out mtensor.nii.gz -mask b0_mask.nii.gz

Comment:
* A good quality brain mask should not remove any brain tissues and keep non-brain tissue at a minimum.
* DTI-TK expects b0_mask.nii.gz to be 0 for the background and 1 for brain tissues.

In [8]:
#! TRY ADDING THIS STEP TO SEE IF ANYTHING IMPROVES

### Make Sure DTI Volumes are SPD
[**NOTE**] Adding this step before norm because norm volume does not process SPD. So, norm will be applied after SPD.
Diffusion tensors, by definition, are symmetric and positive-definite matrices, or SPD. However, in practice, due to noises in the DWI images, the diffusion tensors estimated within a DTI volume are not always SPD. The SPD condition is very important however to ensure DTI volumes behave properly after various image processing steps. DTI-TK provides a simple tool that identifies the voxels that are not SPD and enforces this condition on such voxels. The tool TVtool can be used as follows:

        TVtool -in tensor.nii.gz -spd -out spd.nii.gz

In [9]:
dtitk_spd_path = dtitk_scaled_path.replace("dtitk_scaled.nii.gz", "dtitk_scaled_spd.nii.gz")

# Just make sure from the output that tensor count in non-spd file is zero.
!TVtool -in {dtitk_scaled_path} -spd -out {dtitk_spd_path}

Reading dti_dtitk_scaled.nii.gz ... Done in 0.04149s
Converting dti_dtitk_scaled.nii.gz to be symmetric positive definite ... Done in 0.074475s
Writing dti_dtitk_scaled_nonSPD.nii.gz ... Done in 0.012321s
non-spd tensors count = 0
Writing dti_dtitk_scaled_spd.nii.gz ... Done in 0.185327s


### Make Sure DTI Volumes Contain No Significant Outliers
Outliers often don't have any effect on image registration. However, when the outliers are substantially outside the normal range, they will. A good indicator that the outliers are worthy of concerns is when their existence distorts the mean significantly.

        TVtool -in tensor.nii.gz -norm
which outputs the tensor norm image as tensor_norm.nii.gz, and

        SVtool -in tensor_norm.nii.gz -stats

In [10]:
# Normalize
!TVtool -in {dtitk_spd_path} -norm

dtitk_norm_path = dtitk_spd_path.replace("dtitk_scaled_spd.nii.gz", "dtitk_scaled_spd_norm.nii.gz")

print("Saved @", dtitk_norm_path)


Reading dti_dtitk_scaled_spd.nii.gz ... Done in 0.041808s
Computing the tensor Euclidean norm map ... Done in 0.003954s
Writing dti_dtitk_scaled_spd_norm.nii.gz ... Done in 0.032403s
Saved @ dti_dtitk_scaled_spd_norm.nii.gz


### Check Whether DTI Volumes Share one Common Voxel Space
It's recommended to set all the origin to [0, 0, 0] and this can be done with "TVAdjustVoxelspace".

        TVAdjustVoxelspace -in tensor.nii.gz -origin 0 0 0

In [11]:
# Display basic volume info (it is alerady there, but do include these steps anyway in the registration pipeline)
!VolumeInfo {dtitk_norm_path}

NIFTI Intent Code: NIFTI_INTENT_NONE
NIFTI Orientation Code: LPI
Volume Info of dti_dtitk_scaled_spd_norm.nii.gz
size: 112x112x48, voxel size: 1.60714x1.60714x2.5, origin: [0, 0, 0]


In [13]:
# Visualize
!TVglyphView -in {dtitk_spd_path} -scale 0.001

Reading dti_dtitk_scaled_spd_norm.nii.gz ... Warning: Volume uses unexpected NIfTI intent code
Done in 0.007372s


It is important to resample the volumes that do not share the same voxel spacing as the rest of the volumes. This is usually never an issue unless you are trying to combine data acquired with different protocols. The tool for resampling tensor volumes is TVResample.
It's ideal but not necessary that all the volumes have the same voxel dimensions. This situation may arise when the same protocol is used if the operator decides to optimze the number of slices to acquire. Not having the same voxel dimensions (z dimension in most cases) may reduce the initial bootstrapped template estimate. Visual inspection can be used to verify if this is a serious issue or not.

## SHOULD I DO RESAMPLING? (check back after trying registration for any significant difference.)

## [REGISTRATION AND SPATIAL NORMALIZATION OF DTI VOLUMES](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Registration)

The spatial normalization pipeline includes the following five steps:

1. Preprocessing of the input DTI volumes
2. Bootstrapping the initial DTI template from the input DTI volumes. While Bootstrap with the help of an existing tensor template, step 3 should be skipped. (Which is true in our case. Therefore, we will skip step 3)
3. Rigid alignment with template refinement
4. Affine alignment with template refinement
5. Deformable alignment with template refinement

Where step 1 is already performed above.

#### Step 2. Bootstrapping the initial DTI template from the input DTI volumes

In [ ]:
# template_path = "../../nihpd_templates/nihpd_asym_00-02_mask.nii"
template_path = "ixi_aging_template.nii.gz"

# Create a subject list file
with open("subjects_list.txt", 'w') as wf:
    wf.write(f"{Path(dtitk_spd_path).resolve().absolute()}\n")

In [ ]:
#! [WARNING] NORM files does not work. Use SPD
!dti_template_bootstrap {template_path} subjects_list.txt

In [ ]:
# output affine filepath
dtitk_aff_path = dtitk_spd_path.replace("dtitk_scaled_spd.nii.gz", "dtitk_scaled_spd_aff.nii.gz")

!TVglyphView -in {dtitk_aff_path} -scale 0.001 -view cornonal

#### 4. Affine alignment with template refinement

In [ ]:
!dti_affine_population mean_initial.nii.gz subjects_list.txt EDS 3

In [ ]:
# !TVglyphView -in mean_affine3.nii.gz -scale 0.001

In [ ]:
!TVglyphView -in {dtitk_aff_path} -scale 0.001

#### 5. Deformable alignment with template refinement

In [ ]:
!TVtool -in mean_affine3.nii.gz -tr
!BinaryThresholdImageFilter mean_affine3_tr.nii.gz mask.nii.gz 0.01 100 1 0

In [ ]:
# Really important to check that the mask is appropriate before embarking on the next most time-consuming step.
!dti_diffeomorphic_population mean_affine3.nii.gz subjects_list_aff.txt mask.nii.gz 0.002

In [ ]:
!VolumeInfo mean_affine3.nii.gz

In [ ]:
!VolumeInfo {template_path}